In [4]:
import numpy as np
## Importing TensorFlow API
import tensorflow as tf
import keras
import pickle as pkl
import Vector_creator as Vc
import pandas as pd
import re
import collections
from sklearn.model_selection import train_test_split

In [5]:
train_data = pd.read_csv('New_train_data.csv')
print('\n\n===========================================================')
print('\nReading of Training Phase Done\n')
print('===========================================================\n\n')




Reading of Training Phase Done





In [6]:
y=train_data['Lable'].values 
x=train_data.drop(['URL','Lable'],axis=1).values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)


In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

print(x[0])

[52.  5.  1.  0.  7.  1. 21.  2.  1.  0. 17. 10.  5.  5.  0. 24.  4.  0.
  0.  0.  0.  0. -1. -1. -1. -1.]


START NN

In [5]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=10, activation = "relu"))
ann.add(tf.keras.layers.Dense(units=2, activation = "softmax"))

In [6]:
ann.compile(optimizer = 'rmsprop', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
ann.fit(x_train, y_train, batch_size = 32, epochs = 20)

Epoch 1/20
175/175 [==============================] - 0s 682us/step - loss: 0.3772 - accuracy: 0.8263
Epoch 2/20
175/175 [==============================] - 0s 628us/step - loss: 0.2661 - accuracy: 0.8755
Epoch 3/20
175/175 [==============================] - 0s 605us/step - loss: 0.2328 - accuracy: 0.8918
Epoch 4/20
175/175 [==============================] - 0s 609us/step - loss: 0.2140 - accuracy: 0.9073
Epoch 5/20
175/175 [==============================] - 0s 605us/step - loss: 0.2024 - accuracy: 0.9129
Epoch 6/20
175/175 [==============================] - 0s 609us/step - loss: 0.1942 - accuracy: 0.9175
Epoch 7/20
175/175 [==============================] - 0s 604us/step - loss: 0.1880 - accuracy: 0.9214
Epoch 8/20
175/175 [==============================] - 0s 599us/step - loss: 0.1837 - accuracy: 0.9227
Epoch 9/20
175/175 [==============================] - 0s 627us/step - loss: 0.1796 - accuracy: 0.9230
Epoch 10/20
175/175 [==============================] - 0s 653us/step - loss: 0.176

In [ ]:
ann.save('MODEL')

In [5]:
ann=keras.models.load_model('MODEL')


In [12]:
y_pred = ann.predict(x_test)
y_pred


array([[2.6871765e-03, 9.9731284e-01],
       [1.8064283e-07, 9.9999976e-01],
       [3.6144484e-05, 9.9996388e-01],
       ...,
       [9.6389836e-01, 3.6101617e-02],
       [7.5730586e-01, 2.4269412e-01],
       [4.4450604e-07, 9.9999952e-01]], dtype=float32)

In [8]:
y_pred = y_pred[:,1]
y_pred

array([0.9998554 , 1.        , 0.99994814, ..., 0.02703607, 0.17605278,
       1.        ], dtype=float32)

In [13]:
y_pred = ( y_pred > 0.5 )

print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

ValueError: cannot reshape array of size 2800 into shape (1400,1)

In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
print("Accuracy: ", accuracy_score(y_test, y_pred)*100)

[[627  47]
 [ 75 651]]
Accuracy:  91.28571428571428


In [6]:

while True:
    url = input("Enter URL: ")
    if url == '':
        break
    x=re.search("((http)|(ftp))s?:\/\/[\-.a-zA-Z0-9]{2,}(\.[a-z]{2,})+(\/[\-.a-zA-Z0-9+=?&]*)*",url)
    if(x):
        vec=Vc.Construct_Vector(url)
        vec=np.array(vec)
        vec=vec.reshape(1,-1)
        result = ann.predict(vec)
        result = ( result[:,1] > 0.5 )
        if(result==True):
            print('Probably a malicious URL')
        else:
            
            print('Probably a Benign URL')
    else:
        print("ERROR NOT A VALID URL")


Enter URL: https://github.com/30-seconds/30-seconds-of-code
Probably a Benign URL
Enter URL: http://en.wikipedia.org/wiki/Google
Probably a Benign URL
Enter URL: http://www.projecthoneypot.org/list_of_ips.php
Probably a malicious URL
Enter URL: https://www.youtube.com/feed/subscriptions
Probably a Benign URL
Enter URL: http://bmwusfactory.com/
Probably a malicious URL
Enter URL: http://www.shadowserver.org/wiki/pmwiki.php?n=Services/Reports
Probably a Benign URL
Enter URL: https://www.linkedin.com/feed/
Probably a malicious URL
Enter URL: http://tarapropertiesllc.com/jumbo/LoginVerification.php...
Probably a malicious URL
Enter URL: https://meet.google.com/are-fcaf-swi
Probably a malicious URL


KeyboardInterrupt: Interrupted by user